In [1]:
from ortools.linear_solver import pywraplp
import pandas as pd
def newSolver(name,integer=False):
    return pywraplp.Solver(name,\
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING \
                         if integer else \
                         pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
# Solution의 값 list 구하기
def SolVal(x):
    if type(x) is not list:
        return 0 if x is None \
    else x if isinstance(x,(int,float)) \
           else x.SolutionValue() if x.Integer() is False \
                else int(x.SolutionValue())
    elif type(x) is list:
        return [SolVal(e) for e in x]
# 목적함수의 값 구하기
def ObjVal(x):
    return x.Objective().Value()

if 변수 is None 은 연산자 처럼 작용하여 변수에 대해 연산자 작용해줌.

## Mixing Problem

### 문제 : 특정 영양소 조건들을 만족하며 어떻게 비용을 최소화?

In [2]:
# 데이터생성용 함수
def gen_diet_problem(nb_foods=5, nb_nutrients=4):
    from random import randint,uniform
    data = []
    ranges = [10**randint(2,4) for i in range(nb_nutrients)]
    x = [randint(15,25) for i in range(nb_foods)] # this must be feasible
    MinNutrient = [0]*nb_nutrients
    MaxNutrient = [0]*nb_nutrients
    for food in range(nb_foods):
        nutrients = [randint(10,ranges[i]) for i in range(nb_nutrients)]
        minmax = [randint(0,x[food]),randint(x[food],2*x[food])]
        cost = round(100*uniform(0,10))/100
        v = nutrients+minmax+[cost]
        data.append(v)
    for j in range(nb_nutrients):
        b = sum([x[i]*data[i][j] for i in range(nb_foods)])
        MinNutrient[j] = randint(0,b)
        MaxNutrient[j] = randint(b, 2*b)
    data.append(MinNutrient+['','','',''])
    data.append(MaxNutrient+['','','',''])
    return data

    N_0 N_1 N_2 N_3 Min Max Cost  영양소 4개, 음식 5개의 데이터의 경우 다음과 같은 형태
F_0  
F_1  
F_2  
F_3  
F_4  
Min  
Max

In [3]:
data = gen_diet_problem(5, 4)
data

[[124, 993, 49, 40, 12, 35, 4.05],
 [615, 319, 62, 81, 14, 30, 5.53],
 [794, 588, 56, 81, 1, 24, 9.52],
 [493, 938, 13, 27, 8, 27, 8.22],
 [723, 459, 63, 93, 2, 28, 1.14],
 [19177, 38013, 3192, 1439, '', '', '', ''],
 [84788, 82843, 4957, 10584, '', '', '', '']]

In [4]:
def solve_diet(N):
    s = newSolver('Diet')
    nbF, nbN = len(N) - 2, len(N[0]) - 3
    # 음식의 개수는 Min, Max 이므로 -2, 영양분의 개수는 Min, Max, Cost이므로 -3 을 해주어야 함.
    FMin, FMax, FCost, NMin, NMax = nbN, nbN + 1, nbN + 2, nbF, nbF +1
    # 음식의 Min, Max, Cost 그 다음 Min, Max 에 대한 인덱스 지정.
    f = [s.NumVar(N[i][FMin], N[i][FMax], '') for i in range(nbF)]
    # s.NumVar(최소값, 최대값, notation)
    # 각각의 데이터에 대한 변수 지정
    for j in range(nbN):
        s.Add(N[NMin][j]<=s.Sum([f[i]*N[i][j] for i in range(nbF)]) <= N[NMax][j])
        # 영양분이 특정 범위를 만족하도록 제약조건 추가
    s.Minimize(s.Sum(f[i]*N[i][FCost] for i in range(nbF)))
    rc = s.Solve()
    return rc, ObjVal(s), SolVal(f)

In [5]:
solve_diet(data)

(0, 203.57999999999998, [12.0, 14.0, 1.0, 8.0, 2.0])

두번째 값이 비용(Objective Function),세번째 리스트가 각각 음식의 개수에 대한 리스트가 반환된다.

## Blending Problem

예를들어 일정 제품들이 있고, 제품이 가지고 있는 성질들이 있을때, 이 제품들을 조합하여 특정 수치의 성분을 만족하는 새로운 제품들의 비 조합을 찾는 것을 할 수 있다.  
우리가 흔히 하는 소금물 문제가 이 문제의 비슷한 예시이자 해결 키 포인트 이다.

### 문제 : 특정 수치의 성분, 제품의 성질들을 만족하며 어떻게 최적의 비를 찾을까??

In [6]:
from random import randint

In [7]:
def gen_raw(n):
    R = []
    for i in range(n):
        R.append([randint(80, 99), randint(600, 1000), 0])
    avgr = sum([R[i][0] for i in range(n)]) / float(n)
    for i in range(n):
        p = randint(50,55)+4*R[i][0]/avgr
        R[i][2] = round(p, 2)
    return R

def gen_refined(n):
    R = []
    for i in range(n):
        R.append([randint(82,96), randint(100, 500), randint(600, 20000), 0])
    avgr = sum(R[i][0] for i in range(n)) / float(n)
    for i in range(n):
        p = 61.0+R[i][0]/avgr
        R[i][3] = round(p, 2)
    return R

In [8]:
C = gen_raw(5)

[octane, availability(최대), cost] 이렇게 구성되어있음

In [9]:
D = gen_refined(5)

[octane, min_demand, max_demand, price] 이렇게 구성되어있음

In [10]:
def solve_gas(C, D):
    s = newSolver('Gas Blending Problem')
    nR, nF = len(C), len(D)
    Roc, Rmax, Rcost = 0, 1, 2 # octane, max, cost에 대한 index
    Foc, Fmin, Fmax, Fprice = 0, 1, 2, 3 # octane, min, max, price에 대한 index
    G = [[s.NumVar(0.0, 10000, '')
         for j in range(nF)] for i in range(nR)]
    R = [s.NumVar(0, C[i][Rmax],'') for i in range(nR)]
    F = [s.NumVar(D[j][Fmin], D[j][Fmax], '') for j in range(nF)]
    for i in range(nR):
        s.Add(R[i] == sum(G[i][j] for j in range(nF)))
    for j in range(nF):
        s.Add(F[j] == sum(G[i][j] for i in range(nR)))
    for j in range(nF):
        s.Add(F[j]*D[j][Foc] == 
             s.Sum(G[i][j]*C[i][Roc] for i in range(nR)))
    Cost = s.Sum(R[i]*C[i][Rcost] for i in range(nR))
    Price = s.Sum(F[j]*D[j][Fprice] for j in range(nF))
    s.Maximize(Price-Cost)
    rc = s.Solve()
    return rc, ObjVal(s), SolVal(G)

In [11]:
solve_gas(C, D)

(0,
 24334.75000000001,
 [[0.0, 0.0, 26.56250000000003, 772.3125, 61.12500000000004],
  [804.9230769230753, 2.759160421506888e-13, 0.0, 81.07692307692425, 0.0],
  [0.0, 0.0, 0.0, 631.0, 0.0],
  [0.0, 255.0000000000002, 0.0, 220.12499999999974, 427.87499999999994],
  [67.07692307692464, 0.0, 398.4375, 301.4855769230753, 0.0]])

## Project Management

프로젝트 일정을 짤때 다양한 제약조건들이 따른다.  
핵심은 이론 제약조건들을 만족하며 소요시간이 제일 최소화 되는 조합을 만드는 것이다.

### 문제 : 인력은 무한히 투입가능하다고 할때, 어느정도의 인력과 어느순서로 진행을 하여야 프로젝트를 빨리 끝낼 수 있을까??

In [12]:
def gen_data(n):
    R=[]
    S=0
    for i in range(n):
        RR=[i]                  # Task number
        RR.append(randint(2,8)) # Duration
        P=[]
        for j in range(i):
            if randint(0,1)*randint(0,1):
                P.append(j)
        RR.append(P) 
        R.append(RR)
    return R

In [13]:
D = gen_data(15)
D

[[0, 7, []],
 [1, 7, [0]],
 [2, 6, [0]],
 [3, 6, []],
 [4, 2, []],
 [5, 5, [1, 2]],
 [6, 3, []],
 [7, 2, [4, 5, 6]],
 [8, 5, [2, 3]],
 [9, 8, [5, 6, 7]],
 [10, 8, [1]],
 [11, 3, [0, 1, 3, 9, 10]],
 [12, 4, [6]],
 [13, 4, [1, 3, 4, 6, 9, 11, 12]],
 [14, 5, [1, 3, 13]]]

In [14]:
def solve_model(D):
    s = newSolver('Project Management')
    n = len(D)
    max_t = sum(D[i][1] for i in range(n)) # 최장 걸리는 시간
    t = [s.NumVar(0, max_t, 't[%i]' % i) for i in range(n)]
    Total = s.NumVar(0, max_t, 'Total')
    for i in range(n):
        s.Add(t[i] + D[i][1] <= Total)
        for j in D[i][2]:
            s.Add(t[j] + D[j][1] <= t[i])
    s.Minimize(Total)
    rc = s.Solve()
    return rc, SolVal(Total), SolVal(t)

In [15]:
solve_model(D)

(0,
 41.0,
 [0.0,
  7.0,
  7.0,
  0.0,
  0.0,
  14.0,
  0.0,
  19.0,
  36.0,
  21.0,
  14.0,
  29.0,
  3.0,
  32.0,
  36.0])

## Multi-Stage Models

In [16]:
from random import uniform
def gen_data_content(m,n):
    # Oils down, acids accross (more oils than acids  m > n)
    R=[]
    for i in range(m):
        RR=[]
        P = 100
        for j in range(n-1):
            if P>1:
                acid = randint(1,min(70,P))*randint(0,1)
            else:
                acid = 0
            RR.append(acid)
            P -= acid
        RR.append(P)
        R.append(RR)
    return R
def gen_data_target(C):
    F=[]
    R0,R1=[],[]
    m,n=len(C),len(C[0])
    P=100
    R=[0 for j in range(n)]
    for i in range(m-1):
        if P:
            f=randint(1,min(20,P))
        else:
            f=0
        F.append(f)
        P-=f
        for j in range(n):
            acid = f*C[i][j]
            R[j] += acid
    f=P
    F.append(f)
    for j in range(n):
        acid = f*C[m-1][j]
        R[j] += acid
    for j in range(n):
        R0.append((0.95*R[j]/100.0))
        R1.append((1.05*R[j]/100.0))
    return [R0,R1]
def gen_data_cost(m,k):
    # Oils down, months accross
    R=[]
    for i in range(m):
        RR=[]
        for j in range(k):
            cost = randint(100,200)
            RR.append(cost)
        R.append(RR)
    return R
def gen_data_inventory(m):
    # Oils down
    R=[]
    for i in range(m):
        cost = [randint(0,200)]
        R.append(cost)
    return R

In [17]:
gas_number = 9
acid_number = 7
time_number = 5

In [18]:
data = gen_data_content(gas_number, acid_number)
data # m x n

[[53, 35, 8, 0, 1, 2, 1],
 [42, 0, 0, 24, 0, 0, 34],
 [0, 33, 41, 9, 0, 0, 17],
 [0, 0, 0, 14, 0, 43, 43],
 [0, 20, 64, 0, 7, 0, 9],
 [0, 0, 0, 68, 0, 0, 32],
 [49, 0, 0, 46, 0, 0, 5],
 [0, 55, 0, 0, 0, 24, 21],
 [0, 14, 3, 49, 0, 30, 4]]

D_ij : i번째 oil에 포함되어 있는 j번째 acid의 함량

In [19]:
target = gen_data_target(data)
target # 2 x n

[[12.825,
  16.5015,
  9.414499999999999,
  30.390499999999996,
  0.7125,
  14.515999999999998,
  10.64],
 [14.175, 18.238500000000002, 10.4055, 33.5895, 0.7875, 16.044, 11.76]]

영양분의 목표.  
첫행이 최소값, 두번째 행이 최대값

In [20]:
costs = gen_data_cost(gas_number, time_number)
costs # m x k

[[120, 173, 136, 100, 104],
 [180, 143, 141, 107, 120],
 [139, 122, 103, 143, 116],
 [160, 188, 114, 120, 163],
 [187, 116, 153, 148, 100],
 [140, 140, 189, 140, 198],
 [122, 193, 183, 198, 149],
 [135, 111, 135, 167, 150],
 [160, 121, 110, 129, 187]]

행이 Oil, 열이 시간에 따른 Oil의 가격변화

In [21]:
inventory = gen_data_inventory(gas_number)
inventory # m

[[179], [94], [107], [62], [163], [199], [138], [22], [138]]

초기 창고에 있는 함유량
### 문제의 목적 : Oil을 어떻게 refined하고 blended를 통해 비용을 최소화 할까?

In [22]:
def solve_multi_stage(data, target, costs, inventory, D, SC, SL):
    s = newSolver('Multi-period soap blending problem')
    oils = range(len(data)) # oil의 총 개수
    periods, acids = range(len(costs[0])), range(len(data[0]))
    buy = [[s.NumVar(0, D, '') for _ in periods] for _ in oils]
    blend = [[s.NumVar(0, D, '') for _ in periods] for _ in oils] # 혼합에 이용
    hold = [[s.NumVar(0, D, '') for _ in periods] for _ in oils]
    prod = [s.NumVar(0, D, '') for _ in periods]
    costP = [s.NumVar(0, D*1000, '') for _ in periods] # cost_purchase
    costS = [s.NumVar(0, D*1000, '') for _ in periods] # cost_save
    acid = [[s.NumVar(0, D*D, '') for _ in periods] for _ in acids]
    for i in oils:
        s.Add(hold[i][0] == inventory[i][0]) # 창고에 있는 보유량과 hold하는 양이 같다.
    for j in periods:
        s.Add(prod[j] == sum(blend[i][j] for i in oils)) # 생산량은 blending 하는 양과 같다.
        s.Add(prod[j] >= D)
        if j < periods[-1]:
            for i in oils:
                s.Add(hold[i][j] + buy[i][j] - blend[i][j] == hold[i][j+1])
        s.Add(sum(hold[i][j] for i in oils) >= SL[0])
        s.Add(sum(hold[i][j] for i in oils) <= SL[1])
        for k in acids:
            s.Add(acid[k][j] == sum(blend[i][j]*data[i][k] for i in oils))
            s.Add(acid[k][j] >= target[0][k]*prod[j])
            s.Add(acid[k][j] <= target[1][k]*prod[j])
        s.Add(costP[j] == sum(buy[i][j]*costs[i][j] for i in oils))
        s.Add(costS[j] == sum(hold[i][j]*SC for i in oils))
    cost_product = s.Sum(costP[j] for j in periods)
    cost_storage = s.Sum(costS[j] for j in periods)
    s.Minimize(cost_product + cost_storage)
    rc = s.Solve()
    B, L, H, A = SolVal(buy), SolVal(blend), SolVal(hold), SolVal(acid)
    CP, CS, P = SolVal(costP), SolVal(costS), SolVal(prod)
    
    return rc, ObjVal(s), B, L, H, P, A, CP, CS

In [23]:
demand = 5000
cost = 5
limits = [500, 2000]

In [24]:
solve = solve_multi_stage(data=data, target=target, costs=costs, inventory=inventory, D=demand, SC=cost, SL=limits)
solve

(0,
 2406785.1809491543,
 [[0.0, 0.0, 53.42029633669328, 1433.5367219008756, 0.0],
  [0.0, 85.36000484390931, 671.9872345913504, 509.4655652203238, 0.0],
  [147.22142150628787, 136.78440682897846, 647.0097845743022, 0.0, 0.0],
  [133.66515087286623, 0.0, 0.0, 264.92973892556125, 0.0],
  [345.92857142857144, 1413.7918545374903, 0.0, 0.0, 0.0],
  [61.989108978722555, 237.1843820428578, 0.0, 347.22932639966115, 0.0],
  [3035.4285714285716, 0.0, 0.0, 0.0, 0.0],
  [952.4361268471068, 1305.4329117428056, 0.0, 0.0, 0.0],
  [1221.3310489378728,
   1821.4464400039576,
   3627.5826844976546,
   944.8386475535767,
   0.0]],
 [[0.0, 0.0, 232.42029633669327, 933.5367219008758, 1226.1479306561391],
  [0.0, 179.3600048439093, 671.9872345913504, 509.4655652203238, 0.0],
  [254.22142150628792,
   136.78440682897846,
   309.76891235295153,
   337.24087222135074,
   244.73641150275665],
  [195.6651508728662, 0.0, 0.0, 264.92973892556125, 266.18138050244136],
  [508.9285714285714,
   562.5,
   475.7256719

In [25]:
gas_index = [f'gas_{i}' for i in range(gas_number)]
time_index = [f'month_{i}' for i in range(time_number)]
print(gas_index)
print(time_index)

['gas_0', 'gas_1', 'gas_2', 'gas_3', 'gas_4', 'gas_5', 'gas_6', 'gas_7', 'gas_8']
['month_0', 'month_1', 'month_2', 'month_3', 'month_4']


In [26]:
buy = pd.DataFrame(solve[2])
buy.columns = time_index
buy.index = gas_index
buy

,month_0,month_1,month_2,month_3,month_4
gas_0,0.000000,0.000000,53.420296,1433.536722,0.0
gas_1,0.000000,85.360005,671.987235,509.465565,0.0
gas_2,147.221422,136.784407,647.009785,0.000000,0.0
gas_3,133.665151,0.000000,0.000000,264.929739,0.0
gas_4,345.928571,1413.791855,0.000000,0.000000,0.0
gas_5,61.989109,237.184382,0.000000,347.229326,0.0
gas_6,3035.428571,0.000000,0.000000,0.000000,0.0
gas_7,952.436127,1305.432912,0.000000,0.000000,0.0
gas_8,1221.331049,1821.446440,3627.582684,944.838648,0.0


In [27]:
blend = pd.DataFrame(solve[3])
blend.columns = time_index
blend.index = gas_index
blend

,month_0,month_1,month_2,month_3,month_4
gas_0,0.000000,0.000000,232.420296,933.536722,1226.147931
gas_1,0.000000,179.360005,671.987235,509.465565,0.000000
gas_2,254.221422,136.784407,309.768912,337.240872,244.736412
gas_3,195.665151,0.000000,0.000000,264.929739,266.181381
gas_4,508.928571,562.500000,475.725672,375.566183,333.764581
gas_5,260.989109,237.184382,0.000000,347.229326,973.106203
gas_6,1446.428571,1154.936322,572.063678,0.000000,0.000000
gas_7,974.436127,907.788444,397.644468,0.000000,0.000000
gas_8,1359.331049,1821.446440,2340.389739,2232.031593,1956.063493


In [28]:
hold = pd.DataFrame(solve[4])
hold.columns = time_index
hold.index = gas_index
hold

,month_0,month_1,month_2,month_3,month_4
gas_0,179.0,179.0,179.000000,0.000000,500.0
gas_1,94.0,94.0,0.000000,0.000000,0.0
gas_2,107.0,0.0,0.000000,337.240872,0.0
gas_3,62.0,0.0,0.000000,0.000000,0.0
gas_4,163.0,0.0,851.291855,375.566183,0.0
gas_5,199.0,0.0,0.000000,0.000000,0.0
gas_6,138.0,1727.0,572.063678,0.000000,0.0
gas_7,22.0,0.0,397.644468,0.000000,0.0
gas_8,138.0,0.0,0.000000,1287.192945,0.0


## 최적화를 이용한 패턴분류

In [29]:
def inner(a, b): # 내적 연산자
    s = 0
    for i in range(len(a)):
        s += a[i]*b[i]
    return s

def gen_features(n,m):
  # Generating n vectors of m features linearly separable
    a=gen_hyperplane(m)
    A,B,i=[],[],0
    while len(A) < n:
        x=[randint(-10,10) for _ in range(m)]
        if inner(a[0:m],x) < a[m]-1:
            A.append(x)
    while len(B) < n:
        x=[randint(-10,10) for _ in range(m)]
        if inner(a[0:m],x) > a[m]+1:
            B.append(x)
    return A,B,a

def gen_hyperplane(m):
    return [randint(-10,10) for _ in range(m+1)]

In [30]:
def solve_classification(A, B):
    n, ma, mb = len(A[0]), len(A), len(B)
    s = newSolver('Classification')
    ya = [s.NumVar(0, 99, '') for _ in range(ma)]
    yb = [s.NumVar(0, 99, '') for _ in range(mb)]
    a = [s.NumVar(-99, 99, '') for _ in range(n+1)]
    for i in range(ma):
        s.Add(ya[i] >= a[n]+1-s.Sum(a[j]*A[i][j] for j in range(n)))
    for i in range(mb):
        s.Add(yb[i] >= s.Sum(a[j]*B[i][j] for j in range(n)) - a[n]+1)
    Agap = s.Sum(ya[i] for i in range(ma))
    Bgap = s.Sum(yb[i] for i in range(mb))
    s.Minimize(Agap + Bgap)
    rc = s.Solve()
    
    return rc, ObjVal(s), SolVal(a)

In [31]:
A, B, a = gen_features(12, 2)

In [32]:
A

[[2, 2],
 [-8, 7],
 [6, 2],
 [5, -1],
 [-6, 10],
 [7, 7],
 [6, 7],
 [-7, 9],
 [9, 4],
 [3, 1],
 [0, 5],
 [-1, 3]]

In [33]:
B

[[-9, -1],
 [1, -5],
 [2, -4],
 [-2, -9],
 [-5, 2],
 [0, -8],
 [-9, 6],
 [-6, -3],
 [-6, -4],
 [-3, 0],
 [-9, 0],
 [-9, 3]]

In [34]:
a

[-6, -7, 8]

In [35]:
solve_classification(A, B)

(0, 0.0, [0.7619047619047616, 1.2380952380952377, 1.571428571428571])